# Code Designated for Generating BERTopic Embeddings from NER 

## Load Radgraph Entities 

In [41]:
import torch 
import pandas as pd 

csv_radgraph_path = "../../NER/ner_database.csv"
tensor_radgraph_path = "../../NER/ner_database.pt"

radgraph_tensor = torch.load(tensor_radgraph_path) # already a df 
sample = radgraph_tensor.iloc[0]['extracts']
sample

/var/folders/gd/p6nf_1w11938t89t0bj2r9lc0000gn/T/ipykernel_98904/2697874972.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  radgraph_tensor = torch.load(tensor_radgraph_

{'0': {'text': 'cardiac silhouette size is normal . mediastinal and hilar contours are unremarkable . the pulmonary vasculature is normal . ill - defined parenchymal opacities are noted bilaterally , most pronounced within both lung bases , concerning for multifocal pneumonia . no pleural effusion or pneumothorax is present . there are no acute osseous abnormalities . findings concerning for multifocal pneumonia . followup radiographs after treatment are recommended to ensure resolution of this finding .',
  'entities': {'1': {'tokens': 'cardiac',
    'label': 'Anatomy::definitely present',
    'start_ix': 0,
    'end_ix': 0,
    'relations': []},
   '2': {'tokens': 'silhouette',
    'label': 'Anatomy::definitely present',
    'start_ix': 1,
    'end_ix': 1,
    'relations': [['modify', '1']]},
   '3': {'tokens': 'size',
    'label': 'Anatomy::definitely present',
    'start_ix': 2,
    'end_ix': 2,
    'relations': [['modify', '1']]},
   '4': {'tokens': 'normal',
    'label': 'Observa

In [45]:
radgraph_tensor

,study_id,subject_id,report_path,full_text,examination,indication,technique,comparison,findings,impression,has_comparison,report_length,radgraph_text,extracts
0,57106576,18110461,../data/files/p18/p18110461/s57106576.txt,FINAL REPORT EXAMINATION: CHEST (PA AND LAT) I...,chest (pa and lat),history: [REMOVED]f with cough,chest pa and lateral,[REMOVED],cardiac silhouette size is normal. mediastinal...,findings concerning for multifocal pneumonia. ...,True,645,cardiac silhouette size is normal. mediastinal...,{'0': {'text': 'cardiac silhouette size is nor...
1,52444794,15447063,../data/files/p15/p15447063/s52444794.txt,FINAL REPORT INDICATION: History of left-sided...,NaN,history of left-sided weakness for 12 hours. p...,ap and lateral radiographs of the chest.,NaN,mild cardiomegaly has been stable compared to ...,interval increase in consolidation at the left...,False,806,mild cardiomegaly has been stable compared to ...,{'0': {'text': 'mild cardiomegaly has been sta...
2,58791719,13243522,../data/files/p13/p13243522/s58791719.txt,FINAL REPORT EXAMINATION: Chest radiograph IND...,chest radiograph,[REMOVED]m with cystic fibrosis and fever/coug...,frontal and lateral view.,comparison is made to multiple chest radiograp...,right chest wall port-a-cath ends at the cavoa...,chronic changes of cystic fibrosis as describe...,True,1136,right chest wall port-a-cath ends at the cavoa...,{'0': {'text': 'right chest wall port - a - ca...
3,51779043,11423061,../data/files/p11/p11423061/s51779043.txt,FINAL REPORT HISTORY: Chest pain. TECHNIQUE: U...,NaN,NaN,upright ap view of the chest.,[REMOVED].,low lung volumes limit assessment of the lung ...,low lung volumes limit assessment of the lung ...,True,685,low lung volumes limit assessment of the lung ...,{'0': {'text': 'low lung volumes limit assessm...
4,58785779,15379716,../data/files/p15/p15379716/s58785779.txt,"FINAL REPORT INDICATION: Pneumonia, cough, and...",NaN,"pneumonia, cough, and shortness of breath.",NaN,chest radiograph from [REMOVED] and chest ct f...,NaN,stable lingular and increased right middle lob...,True,600,nan,"{'0': {'text': 'nan', 'entities': {}, 'data_so..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2754,58255867,14236258,../data/files/p14/p14236258/s58255867.txt,FINAL REPORT INDICATION: [REMOVED]M with cough...,NaN,[REMOVED]m with cough // r/o pna,ap and lateral views of the chest.,[REMOVED]. [REMOVED] and ct torso from [REMOVED].,vague opacity projecting over the right mid/lo...,"vague right mid/lower opacity, nonspecific the...",True,695,vague opacity projecting over the right mid/lo...,{'0': {'text': 'vague opacity projecting over ...
2755,59289169,10623647,../data/files/p10/p10623647/s59289169.txt,WET READ: [REMOVED] [REMOVED] [REMOVED] 4:26 P...,chest: frontal and lateral views,history: [REMOVED]m with hypoxia // pna?,chest: frontal and lateral,[REMOVED],bilateral patchy pulmonary opacities appear sl...,patchy bilateral mid to lower lung opacities a...,True,1760,bilateral patchy pulmonary opacities appear sl...,{'0': {'text': 'bilateral patchy pulmonary opa...
2756,55182265,15116068,../data/files/p15/p15116068/s55182265.txt,WET READ: [REMOVED] [REMOVED] [REMOVED] 11:56 ...,chest radiograph.,"[REMOVED]f with cough, fever, sob // ? pna",single portable semi upright radiograph the ch...,"chest ct: [REMOVED], [REMOVED]",extensive bronchiectasis is again noted in the...,"extensive bilateral bronchiectasis, with super...",True,1100,extensive bronchiectasis is again noted in the...,{'0': {'text': 'extensive bronchiectasis is ag...
2757,50696726,17025867,../data/files/p17/p17025867/s50696726.txt,FINAL REPORT INDICATION: Cough and weakness. C...,NaN,cough and weakness.,NaN,radiograph available from [REMOVED]. frontal a...,NaN,1. increase in density of a right lower and mi...,True,649,nan,"{'0': {'text': 'nan', 'entities': {}, 'data_so..."


# Processing Code 

## BERTopic Using RADGRAPH ENTITIES ONLY 

In [46]:
import json

def extract_radgraph_entities(radgraph_dict):
    entities = json.loads(radgraph_dict)
    return [e["text"] for e in entities["entities"]]

## BERTopic Using RADGRAPH ENTITIES AND RELATIONS 

In [50]:
import json
import pandas as pd

def process_radgraph_extracts(extracts_json):
    """
    Converts RadGraph 'extracts' into structured clinical text for BERTopic.

    Args:
    - extracts_json (str or dict): RadGraph JSON stored in the 'extracts' column.

    Returns:
    - str: Reformatted text with preserved relations.
    """
    if isinstance(extracts_json, str):
        extracts_json = json.loads(extracts_json)  # Convert string to dict if needed

    # Extract only the first key (assumed report ID or study ID)
    first_key = list(extracts_json.keys())[0]
    radgraph_data = extracts_json[first_key]

    entities = radgraph_data.get("entities", {})
    sentences = []
    
    entity_map = {eid: entity["tokens"] for eid, entity in entities.items()}  # ID → Text map

    # Convert relations into readable text
    for eid, entity in entities.items():
        entity_text = entity["tokens"]
        entity_label = entity["label"].split("::")[0]  # Keep main category (Anatomy, Observation)
        relations = entity.get("relations", [])

        # Base description of the entity
        entity_sentence = f"{entity_text} is an {entity_label.lower()}."
        
        # Add relational information
        for rel in relations:
            relation_type, target_eid = rel
            target_text = entity_map.get(target_eid, "")
            
            if relation_type == "modify":
                entity_sentence = f"{entity_text} modifies {target_text}."
            elif relation_type == "located_at":
                entity_sentence = f"{entity_text} is located at {target_text}."
            elif relation_type == "associated_with":
                entity_sentence = f"{entity_text} is associated with {target_text}."
        
        sentences.append(entity_sentence)

    return " ".join(sentences)

# Apply processing to the entire 'extracts' column
# Apply to a sample (radgraph extract for the first study)
sample_extract = process_radgraph_extracts(radgraph_tensor['extracts'][0])
print(sample_extract)

radgraph_tensor["processed_radgraph"] = radgraph_tensor["extracts"].apply(process_radgraph_extracts)
radgraph_tensor


cardiac is an anatomy. silhouette modifies cardiac. size modifies cardiac. normal is located at cardiac. mediastinal is an anatomy. hilar is an anatomy. contours modifies hilar. unremarkable is located at hilar. pulmonary modifies vasculature. vasculature is an anatomy. normal is located at vasculature. ill - defined modifies opacities. parenchymal is an anatomy. opacities is located at lung. bilaterally modifies parenchymal. pronounced is an observation. both modifies lung. lung is an anatomy. bases modifies lung. multifocal modifies pneumonia. pneumonia is an observation. pleural is an anatomy. effusion is located at pleural. pneumothorax is an observation. acute modifies abnormalities. osseous is an anatomy. abnormalities is located at osseous. multifocal modifies pneumonia. pneumonia is an observation.


In [53]:
radgraph_tensor.to_csv('./processed_NER_extracts/processed_NER_extracts.csv')
torch.save(radgraph_tensor, './processed_NER_extracts/processed_NER_extracts.pt')

# BERTopic Radgraph - Document Representations

In [1]:
import ipykernel
ipykernel.__version__
import scipy
from scipy import stats

import pandas as pd
import json

# NLTK 
import re
import nltk
import umap.umap_ as UMAP
from nltk import word_tokenize        
nltk.download('wordnet')  
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
import pickle 
from bertopic.representation import MaximalMarginalRelevance
import torch

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/JanayeCheong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import numpy 

# Path to the pre-trained model used for embeddings
model_path = "microsoft/BiomedVLP-CXR-BERT-specialized"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
embedding_model = AutoModel.from_pretrained(model_path, trust_remote_code=True)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.preprocessing import normalize

class NER_Extracts_Embedder:
    def __init__(self, pretrained_model_name, pooling_strategy='cls'):
        """
        Initialize the RadiologyReportEmbedder with a specified pooling strategy.

        Parameters:
            pretrained_model_name: Name of the pretrained model to load
            pooling_strategy: Pooling strategy ('mean' or 'cls')
        """
        self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name, trust_remote_code=True)
        self.model = AutoModel.from_pretrained(pretrained_model_name, trust_remote_code=True)
        self.model.eval()  # Set to evaluation mode
        self.pooling_strategy = pooling_strategy.lower()  # Ensure case-insensitivity
        
        if self.pooling_strategy not in ['mean', 'cls']:
            raise ValueError("Invalid pooling strategy! Use 'mean' or 'cls'.")

    def create_embeddings(self, texts):
        """
        Create embeddings for a list of radiology report texts.

        Parameters:
            texts: List of report texts to embed

        Returns:
            Normalized embeddings as a numpy array
        """
        embeddings = []
        batch_size = 32
        
        with torch.no_grad():
            for i in range(0, len(texts), batch_size):
                batch_texts = texts[i:i + batch_size]
                
                inputs = self.tokenizer(
                    batch_texts,
                    padding=True,
                    truncation=True,
                    max_length=512,  # Adjust max length if needed
                    return_tensors="pt"
                )
                
                outputs = self.model(**inputs)
                
                # Apply the selected pooling strategy
                if self.pooling_strategy == 'cls':
                    # Use [CLS] token embedding
                    batch_embeddings = outputs.last_hidden_state[:, 0, :]
                else:  # Mean pooling
                    attention_mask = inputs['attention_mask']
                    batch_embeddings = self._mean_pooling(outputs.last_hidden_state, attention_mask)
                
                embeddings.append(batch_embeddings.numpy())
        
        embeddings = np.vstack(embeddings)
        embeddings = normalize(embeddings)
        
        # Save the model for the specific pooling strategy
        model_save_path = f'../../models/CXR_BERT_{self.pooling_strategy}_model'
        self.model.save_pretrained(model_save_path)
        print(f"Model saved at: {model_save_path}")
        
        return embeddings

    def _mean_pooling(self, token_embeddings, attention_mask):
        """
        Perform mean pooling on token embeddings using the attention mask.
        """
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [ ]:
def process_pneumonia_reports(reports_df, pretrained_model_name, pooling_strategy='mean'):
    """
    Process pneumonia reports and create embeddings with the specified pooling strategy.

    Parameters:
        reports_df: DataFrame containing the radiology reports
        pretrained_model_name: Name of the pretrained model to use
        pooling_strategy: Pooling strategy ('mean' or 'cls')

    Returns:
        DataFrame with embeddings added
    """
    # Initialize the embedder
    embedder = RadiologyReportEmbedder(pretrained_model_name, pooling_strategy=pooling_strategy)
    
    # Combine relevant sections for embedding
    reports_df['embedding_text'] = reports_df.apply(
        lambda x: f"Findings: {x['findings']} Impression: {x['impression']}", 
        axis=1
    )
    
    # Create and add embeddings
    embeddings = embedder.create_embeddings(reports_df['embedding_text'].tolist())
    torch.save(embeddings, f'radiology_embeddings_{pooling_strategy}.pt')
    print(f"Embeddings saved to: radiology_embeddings_{pooling_strategy}.pt")
    reports_df['embedding'] = embeddings.tolist()
    
    return reports_df

In [ ]:
reports_df = pd.read_csv('structured_reports.csv')
embedded_reports_df = process_pneumonia_reports(reports_df, pretrained_model_name='microsoft/BiomedVLP-CXR-BERT-specialized')
embedded_reports_df.to_csv('embedded_reports_cls.csv')
embeddings = embedded_reports_df['embedding'].tolist()
torch.save(embeddings, 'radiology_embeddings_cls.pt')

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from bertopic import BERTopic
import umap
import hdbscan

class ClinicalOutcomePredictor:
    def __init__(self):
        self.outcomes = ['icu_stay', 'ventilator', 'expired', 'sepsis', 'severe']
        self.models = {outcome: {} for outcome in self.outcomes}
        
    def process_radgraph_to_structured(self, radgraph_extracts):
        """Convert RadGraph extracts to structured format"""
        structured_texts = []
        for extract in radgraph_extracts:
            # Convert relationships to structured sentences
            structured = []
            entities = {}
            relations = []
            
            # Parse each statement
            for statement in extract.split('. '):
                if 'is an' in statement:
                    entity, type_ = statement.split(' is an ')
                    entities[entity] = type_
                elif 'modifies' in statement:
                    modifier, target = statement.split(' modifies ')
                    relations.append(f"{modifier}_modifies_{target}")
                elif 'located at' in statement:
                    finding, location = statement.split(' is located at ')
                    relations.append(f"{finding}_at_{location}")
                elif 'associated with' in statement:
                    source, target = statement.split(' is associated with ')
                    relations.append(f"{source}_associates_{target}")
            
            # Create structured representation
            structured_text = " ".join([
                f"{entity}_{type_}" for entity, type_ in entities.items()
            ] + relations)
            
            structured_texts.append(structured_text)
            
        return structured_texts

    def create_bertopic_model(self, texts, n_topics=50):
        """Create and train BERTopic model"""
        # Initialize BERTopic with custom parameters
        topic_model = BERTopic(
            n_topics=n_topics,
            min_topic_size=5,
            umap_model=umap.UMAP(
                n_neighbors=15,
                n_components=5,
                min_dist=0.0,
                metric='cosine'
            ),
            hdbscan_model=hdbscan.HDBSCAN(
                min_cluster_size=5,
                metric='euclidean',
                cluster_selection_method='eom'
            )
        )
        
        # Fit the model and transform documents
        topics, probs = topic_model.fit_transform(texts)
        return topic_model, probs

    def train_outcome_models(self, 
                           raw_reports,
                           radgraph_extracts,
                           outcome_labels,
                           embedder="microsoft/BiomedVLP-CXR-BERT-general"):
        """
        Train models using different text representations
        """
        # 1. Process raw embeddings
        tokenizer = AutoTokenizer.from_pretrained(embedder)
        model = AutoModel.from_pretrained(embedder)
        
        raw_embeddings = []
        for report in raw_reports:
            inputs = tokenizer(report, return_tensors="pt", padding=True, truncation=True)
            with torch.no_grad():
                outputs = model(**inputs)
            # Use [CLS] token embedding
            raw_embeddings.append(outputs.last_hidden_state[:, 0, :].numpy())
        raw_embeddings = np.vstack(raw_embeddings)
        
        # 2. Process structured RadGraph texts
        structured_texts = self.process_radgraph_to_structured(radgraph_extracts)
        structured_embeddings = []
        for text in structured_texts:
            inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
            with torch.no_grad():
                outputs = model(**inputs)
            structured_embeddings.append(outputs.last_hidden_state[:, 0, :].numpy())
        structured_embeddings = np.vstack(structured_embeddings)
        
        # 3. Create BERTopic representations
        topic_model, topic_probs = self.create_bertopic_model(structured_texts)
        
        # Train models for each outcome
        results = {}
        for outcome in self.outcomes:
            outcome_labels = outcome_labels[outcome]
            
            # Train models using different representations
            representations = {
                'raw_embeddings': raw_embeddings,
                'structured_embeddings': structured_embeddings,
                'topic_probs': topic_probs
            }
            
            outcome_results = {}
            for rep_name, features in representations.items():
                # Split data
                X_train, X_test, y_train, y_test = train_test_split(
                    features, outcome_labels, test_size=0.2, random_state=42
                )
                
                # Train model
                model = RandomForestClassifier(n_estimators=100, random_state=42)
                model.fit(X_train, y_train)
                
                # Evaluate
                y_pred = model.predict(X_test)
                y_pred_proba = model.predict_proba(X_test)[:, 1]
                
                metrics = {
                    'classification_report': classification_report(y_test, y_pred),
                    'roc_auc': roc_auc_score(y_test, y_pred_proba)
                }
                
                # Store model and results
                self.models[outcome][rep_name] = model
                outcome_results[rep_name] = metrics
            
            results[outcome] = outcome_results
        
        return results, topic_model

    def analyze_feature_importance(self, outcome, representation):
        """Analyze which features are most important for prediction"""
        model = self.models[outcome][representation]
        
        if representation == 'topic_probs':
            # Get topic interpretations
            important_topics = pd.DataFrame({
                'topic_id': range(len(model.feature_importances_)),
                'importance': model.feature_importances_
            }).sort_values('importance', ascending=False)
            
            return important_topics
        else:
            # For embeddings, we can only show importance scores
            feature_importance = pd.DataFrame({
                'feature_id': range(len(model.feature_importances_)),
                'importance': model.feature_importances_
            }).sort_values('importance', ascending=False)
            
            return feature_importance

    def predict_outcomes(self, new_reports, new_radgraph_extracts):
        """Predict outcomes for new reports using all models"""
        # Process new reports similar to training
        # Return predictions from each model
        pass